In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Loading dataset
df = pd.read_csv("data.csv")

# Loading MiniLM
embedder = SentenceTransformer('all-MiniLM-L6-v2')
prompt_embeddings = embedder.encode(df['prompt'].tolist(), convert_to_tensor=True)

# Loading non-GPT language model
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)




Device set to use cuda:0


In [5]:
def get_best_match(user_input):
    user_embedding = embedder.encode(user_input, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(user_embedding, prompt_embeddings)
    best_idx = similarities.argmax().item()  # Fixed here
    return df['prompt'].iloc[best_idx], df['response'].iloc[best_idx]
def generate_response(user_input):
    matched_prompt, matched_response = get_best_match(user_input)

    prompt = f"""User question: {user_input}
Related prior question: {matched_prompt}
Answer to prior question: {matched_response}
Now, provide a creative and informative response to the user's question:"""


    output = generator(
        prompt,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        top_p=0.9
    )[0]['generated_text']

    return output.split("Now, provide a creative and informative response to the user's question:")[-1].strip()

In [6]:
# Main Function
if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        print("Bot:", generate_response(user_input))


You: Who invented Light Bulb


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: Who invented Light Bulb?
Answer: Thomas Edison is credited with inventing the first commercially practicable incandescent light bulb in 1879. He initially developed an incandescent light that lasted 14 hours before going out, but he gradually improved the design and by 1889, he was able to produce a bulb that could last 1,200 hours.
(10) comments
Thomas Edison is credited with inventing the first commercially practicable incandescent light bulb in 1879. He initially developed an
You: Explain the story of Romeo and Juliet


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: Question: Explain the story of Romeo and Juliet.
Answer:
In the famous play Romeo and Juliet, two families, the Capulets and the Montagues, fight for the affections of a young woman named Juliet. The two families are feuding, and the feud is made worse when the families' children are engaged to be married. The feuding families, however, find that their feud has made things worse. The feud has made them enemies, and they are willing to kill
You: Exit
